In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Load training data
data = pd.read_csv('train.csv')

# Tokenize and encode sequences with separator token
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Claim'] + ' [SEP] ' + data['Evidence'])
X = tokenizer.texts_to_sequences(data['Claim'] + ' [SEP] ' + data['Evidence'])
X = pad_sequences(X) # Pad sequences to ensure uniform length
y = data['label'] # Assign labels to y variable

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define function to build model with hyperparameters
def build_model(hp):
    model = Sequential()
    # Add an embedding layer
    model.add(Embedding(len(tokenizer.word_index) + 1, hp.Int('embedding_dim', min_value=50, max_value=150, step=50), input_length=X_train.shape[1]))
    # Add bidirectional LSTM layers
    model.add(Bidirectional(LSTM(hp.Int('lstm_unit1', min_value=32, max_value=96, step=32), return_sequences=True)))
    model.add(Bidirectional(LSTM(hp.Int('lstm_unit2', min_value=32, max_value=96, step=32))))
    # Add dropout layer
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)))
    # Add a dense layer with sigmoid activation for binary classification and apply L2 regularization
    model.add(Dense(1, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(hp.Float('l2_regularization', min_value=1e-6, max_value=1e-4, sampling='log'))))
    # Compile the model with binary cross-entropy loss and Adam optimizer
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Instantiate the RandomSearch tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='hyp',
    project_name='bilstm_hyperparameter_tuning'
)

# Search for the best hyperparameters
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=3)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Save the best model
best_model.save('best_bilstm_model.h5')


Trial 3 Complete [00h 21m 31s]
val_accuracy: 0.8196582794189453

Best val_accuracy So Far: 0.8196582794189453
Total elapsed time: 01h 16m 33s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
100               |150               |embedding_dim
32                |64                |lstm_unit1
96                |32                |lstm_unit2
0.2               |0.2               |dropout
8.2216e-06        |1.444e-06         |l2_regularization

Epoch 1/3
199/593 [=========>....................] - ETA: 4:15 - loss: 0.5213 - accuracy: 0.7564

KeyboardInterrupt: 

In [ ]:
# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hp.values)

# Build model with best hyperparameters
model = build_model(best_hp)
# Fit model to training data
model.fit(X_train,y_train, epochs=1, validation_data = ([X_val, y_val]))
# Save the model
model.save('best_model.h5')

Best Hyperparameters:
{'embedding_dim': 150, 'lstm_unit1': 64, 'lstm_unit2': 32, 'dropout': 0.2, 'l2_regularization': 1.4440277733746057e-06}
593/593 [==============================] - 475s 789ms/step - loss: 0.4714 - accuracy: 0.7764 - val_loss: 0.4096 - val_accuracy: 0.8159


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Import necessary libraries for evaluation
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Load development data
dev_data = pd.read_csv('dev.csv')

# Tokenize and encode sequences
X_dev = tokenizer.texts_to_sequences(dev_data['Claim'] + ' [SEP] ' + dev_data['Evidence'])
X_dev = pad_sequences(X_dev, maxlen=308)
y_dev = dev_data['label']

# Load trained model
model = load_model('best_model.h5')

# Evaluate model on development set
loss, accuracy = model.evaluate(X_dev, y_dev)
print("Loss:", loss)
print("Accuracy:", accuracy)

186/186 [==============================] - 36s 182ms/step - loss: 0.4178 - accuracy: 0.8041
Loss: 0.41780468821525574
Accuracy: 0.8040837049484253


In [ ]:
# Import necessary libraries for evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions
y_pred = model.predict(X_dev)
y_pred = np.round(y_pred)
# Calculate precision, recall, and F1 score
precision = precision_score(y_dev, y_pred)
recall = recall_score(y_dev, y_pred)
f1 = f1_score(y_dev, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

186/186 [==============================] - 29s 158ms/step
Precision: 0.7062146892655368
Recall: 0.46904315196998125
F1 Score: 0.5636978579481399
